In [1]:
from ast import literal_eval
from os import listdir
from os.path import isfile, join
from scipy.sparse import csr_matrix, load_npz, save_npz
from tqdm import tqdm

import seaborn as sns
import datetime
import json
import numpy as np
import pandas as pd
import time
import yaml
import scipy.sparse as sparse
from ast import literal_eval

In [2]:
df_train = pd.read_csv('CDsVinyl/Train.csv')
df_valid = pd.read_csv('CDsVinyl/Valid.csv')
df_test = pd.read_csv('CDsVinyl/Test.csv')

In [3]:
rtrain = load_npz('CDsVinyl/Rtrain.npz')
rvalid = load_npz('CDsVinyl/Rvalid.npz')
rtest = load_npz('CDsVinyl/Rtest.npz')

In [6]:
keyphrases = pd.read_csv('CDsVinyl/KeyPhrases.csv')

In [5]:
df_train.head()

,reviewerID,asin,overall,unixReviewTime,reviewText,review,conca_review,keyVector,keyphrases_indices_length,UserIndex,ItemIndex,Binary
0,A2GANR9I6XHTU9,0780018664,4.0,1169337600,Filmed in 1931 this is the first serial killer...,"['filmed', 'first', 'serial', 'killer', 'movie...",filmed first serial killer movie story serial...,[],0,2353,0,1
1,ANCOMAI0I7LVG,0780018664,5.0,1213574400,My feelings for this iconic piece of filmmakin...,"['feeling', 'iconic', 'piece', 'filmmaking', '...",feeling iconic piece filmmaking mirror previo...,[12],1,5482,0,1
2,A1GGOC9PVDXW7Z,0780018664,5.0,1268524800,The title M comes from a chalk mark (for 'murd...,"['title', 'come', 'chalk', 'mark', 'murderer',...",title come chalk mark murderer placed beggar ...,[12],1,755,0,1
3,A62G4QX6XQVLP,0780018664,5.0,1165190400,This film is easily in the Top 5 of Fritz Lang...,"['film', 'easily', 'top', 'fritz', 'lang', 'be...",film easily top fritz lang best film quite po...,"[12, 33]",2,4781,0,1
4,A2ILOYARQVO4K1,0780018664,4.0,989193600,"Dark, disturbingly satirical, humorous exactly...","['dark', 'disturbingly', 'satirical', 'humorou...",dark disturbingly satirical humorous exactly ...,[5],1,2466,0,1


In [9]:
rtrain

<6056x4395 sparse matrix of type '<type 'numpy.float32'>'
	with 97781 stored elements in Compressed Sparse Row format>

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
def train(matrix_train):
    similarity = cosine_similarity(X=matrix_train, Y=None, dense_output=True)
    return similarity

def get_I_K(df, row_name = 'ItemIndex', shape = (3668,75)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(df.shape[0])):
        key_vector = literal_eval(df['keyVector'][i])
        rows.extend([df[row_name][i]]*len(key_vector)) ## Item index
        cols.extend(key_vector) ## Keyword Index
#         if binary:
        vals.extend(np.array([1]*len(key_vector)))
#         else:
#             vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)

def predict(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
    
    if item_similarity_en:
        matrix_train = matrix_train.transpose()
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        vector_u = similarity[user_index]
        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    if item_similarity_en:
        res = res.transpose()
    return res

def prediction(prediction_score, topK, matrix_Train):

    prediction = []

    for user_index in tqdm(range(matrix_Train.shape[0])):
        vector_u = prediction_score[user_index]
        vector_train = matrix_Train[user_index]
        if len(vector_train.nonzero()[0]) > 0:
            vector_predict = sub_routine(vector_u, vector_train, topK=topK)
        else:
            vector_predict = np.zeros(topK, dtype=np.float32)

        prediction.append(vector_predict)

    return np.vstack(prediction)


def sub_routine(vector_u, vector_train, topK=500):

    train_index = vector_train.nonzero()[1]

    vector_u = vector_u

    candidate_index = np.argpartition(-vector_u, topK+len(train_index))[:topK+len(train_index)]
    vector_u = candidate_index[vector_u[candidate_index].argsort()[::-1]]
    vector_u = np.delete(vector_u, np.isin(vector_u, train_index).nonzero()[0])

    return vector_u[:topK]

def predict2(matrix_train, k, similarity, item_similarity_en = False):
    prediction_scores = []
        
    for user_index in tqdm(range(matrix_train.shape[0])):
        # Get user u's prediction scores for all items
        vector_u = similarity[user_index]

        # Get closest K neighbors excluding user u self
        similar_users = vector_u.argsort()[::-1][1:k+1]
        # Get neighbors similarity weights and ratings
        similar_users_weights = similarity[user_index][similar_users]
        similar_users_ratings = matrix_train[similar_users].toarray()

        prediction_scores_u = similar_users_ratings * similar_users_weights[:, np.newaxis]

        prediction_scores.append(np.sum(prediction_scores_u, axis=0))
    res = np.array(prediction_scores)
    if item_similarity_en:
        res = res.transpose()
    return res


In [16]:
I_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (4395,40))

100%|███████████████████████████████████████████████████████████████████████| 116660/116660 [00:04<00:00, 25617.04it/s]


In [17]:
U_K = get_I_K(df_train, row_name = 'ItemIndex', shape = (6056,40))

100%|███████████████████████████████████████████████████████████████████████| 116660/116660 [00:04<00:00, 24529.02it/s]


In [18]:
save_npz('CDsVinyl/I_K',I_K)
save_npz('CDsVinyl/U_K',U_K)

In [19]:
def recallk(vector_true_dense, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_true_dense)


def precisionk(vector_predict, hits, **unused):
    hits = len(hits.nonzero()[0])
    return float(hits)/len(vector_predict)


def average_precisionk(vector_predict, hits, **unused):
    precisions = np.cumsum(hits, dtype=np.float32)/range(1, len(vector_predict)+1)
    return np.mean(precisions)


def r_precision(vector_true_dense, vector_predict, **unused):
    vector_predict_short = vector_predict[:len(vector_true_dense)]
    hits = len(np.isin(vector_predict_short, vector_true_dense).nonzero()[0])
    return float(hits)/len(vector_true_dense)


def _dcg_support(size):
    arr = np.arange(1, size+1)+1
    return 1./np.log2(arr)


def ndcg(vector_true_dense, vector_predict, hits):
    idcg = np.sum(_dcg_support(len(vector_true_dense)))
    dcg_base = _dcg_support(len(vector_predict))
    dcg_base[np.logical_not(hits)] = 0
    dcg = np.sum(dcg_base)
    return dcg/idcg


def click(hits, **unused):
    first_hit = next((i for i, x in enumerate(hits) if x), None)
    if first_hit is None:
        return 5
    else:
        return first_hit/10


def evaluate(matrix_Predict, matrix_Test, metric_names =['R-Precision', 'NDCG', 'Precision', 'Recall', 'MAP'], atK = [5, 10, 15, 20, 50], analytical=False):
    """
    :param matrix_U: Latent representations of users, for LRecs it is RQ, for ALSs it is U
    :param matrix_V: Latent representations of items, for LRecs it is Q, for ALSs it is V
    :param matrix_Train: Rating matrix for training, features.
    :param matrix_Test: Rating matrix for evaluation, true labels.
    :param k: Top K retrieval
    :param metric_names: Evaluation metrics
    :return:
    """
    global_metrics = {
        "R-Precision": r_precision,
        "NDCG": ndcg,
        "Clicks": click
    }

    local_metrics = {
        "Precision": precisionk,
        "Recall": recallk,
        "MAP": average_precisionk
    }

    output = dict()

    num_users = matrix_Predict.shape[0]

    for k in atK:

        local_metric_names = list(set(metric_names).intersection(local_metrics.keys()))
        results = {name: [] for name in local_metric_names}
        topK_Predict = matrix_Predict[:, :k]

        for user_index in tqdm(range(topK_Predict.shape[0])):
            vector_predict = topK_Predict[user_index]
            if len(vector_predict.nonzero()[0]) > 0:
                vector_true = matrix_Test[user_index]
                vector_true_dense = vector_true.nonzero()[1]
                hits = np.isin(vector_predict, vector_true_dense)

                if vector_true_dense.size > 0:
                    for name in local_metric_names:
                        results[name].append(local_metrics[name](vector_true_dense=vector_true_dense,
                                                                 vector_predict=vector_predict,
                                                                 hits=hits))

        results_summary = dict()
        if analytical:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = results[name]
        else:
            for name in local_metric_names:
                results_summary['{0}@{1}'.format(name, k)] = (np.average(results[name]),
                                                              1.96*np.std(results[name])/np.sqrt(num_users))
        output.update(results_summary)

    global_metric_names = list(set(metric_names).intersection(global_metrics.keys()))
    results = {name: [] for name in global_metric_names}

    topK_Predict = matrix_Predict[:]

    for user_index in tqdm(range(topK_Predict.shape[0])):
        vector_predict = topK_Predict[user_index]

        if len(vector_predict.nonzero()[0]) > 0:
            vector_true = matrix_Test[user_index]
            vector_true_dense = vector_true.nonzero()[1]
            hits = np.isin(vector_predict, vector_true_dense)

            # if user_index == 1:
            #     import ipdb;
            #     ipdb.set_trace()

            if vector_true_dense.size > 0:
                for name in global_metric_names:
                    results[name].append(global_metrics[name](vector_true_dense=vector_true_dense,
                                                              vector_predict=vector_predict,
                                                              hits=hits))

    results_summary = dict()
    if analytical:
        for name in global_metric_names:
            results_summary[name] = results[name]
    else:
        for name in global_metric_names:
            results_summary[name] = (np.average(results[name]), 1.96*np.std(results[name])/np.sqrt(num_users))
    output.update(results_summary)

    return output


In [26]:
user_item_res = get_res(rtrain,rvalid, k = 100, base = 0)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5180.50it/s]


In [28]:
item_based_res = get_res(rtrain,rvalid, k = 100, I_K = I_K, base = 1)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5207.22it/s]


In [30]:
user_based_res = get_res(rtrain,rvalid, k = 100, U_K = U_K, base = 2)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5321.62it/s]


In [27]:
user_item_res

{'MAP@10': (0.06168417231301177, 0.002846614511429719),
 'MAP@15': (0.05547976387010092, 0.002346142671208233),
 'MAP@20': (0.05093969639491765, 0.0020347192790403914),
 'MAP@5': (0.07089213570203111, 0.0038712470885525103),
 'MAP@50': (0.03694294299260082, 0.001273152056150982),
 'NDCG': (0.1588749840350779, 0.004723496258055915),
 'Precision@10': (0.04793292379213016, 0.0018987655866329243),
 'Precision@15': (0.04035641153356578, 0.0014824463457221862),
 'Precision@20': (0.03545575294703636, 0.001242405336155152),
 'Precision@5': (0.06259339199734351, 0.002923707056099553),
 'Precision@50': (0.02293541424539266, 0.000740451049508006),
 'R-Precision': (0.06350889522349694, 0.003660306124425233),
 'Recall@10': (0.14197793609017711, 0.006059166090170213),
 'Recall@15': (0.17695890434443642, 0.006699604061189737),
 'Recall@20': (0.20409379235786, 0.007038388448278041),
 'Recall@5': (0.09501616202214352, 0.00503834384528376),
 'Recall@50': (0.31421806458121376, 0.008115025763460492)}

In [29]:
item_based_res

{'MAP@10': (0.01866572978186792, 0.001552326498395624),
 'MAP@15': (0.017663077325205834, 0.001288537079048407),
 'MAP@20': (0.016941324590700704, 0.0011249645028732973),
 'MAP@5': (0.02022967513420776, 0.002089026801829122),
 'MAP@50': (0.014320965320317205, 0.0007266383553107286),
 'NDCG': (0.06618155222899696, 0.0029192558408785785),
 'Precision@10': (0.016237755271459407, 0.0011011110155266521),
 'Precision@15': (0.015263711328795174, 0.0008802375409049988),
 'Precision@20': (0.014394819857214013, 0.0007546292783539829),
 'Precision@5': (0.018562178316453598, 0.0016019083775005698),
 'Precision@50': (0.011386352316121534, 0.0004823549607395219),
 'R-Precision': (0.01903772782640183, 0.0019929563994508727),
 'Recall@10': (0.04635101287756888, 0.003543237586201303),
 'Recall@15': (0.06522716006915662, 0.004121350055580288),
 'Recall@20': (0.0806489519720455, 0.004519378555520585),
 'Recall@5': (0.02694176586685269, 0.002712117485177165),
 'Recall@50': (0.156427718994557, 0.0062129282

In [31]:
user_based_res

{'MAP@10': (0.0043223068185711385, 0.0007359233379531491),
 'MAP@15': (0.004069412855731967, 0.0006097757112980155),
 'MAP@20': (0.003871958420480238, 0.0005324633690017823),
 'MAP@5': (0.004716365045104876, 0.0010089453804348748),
 'MAP@50': (0.003258456267190244, 0.0003377468476830535),
 'NDCG': (0.013348753977931302, 0.001243502789898573),
 'Precision@10': (0.003685870828490786, 0.0005046618208904838),
 'Precision@15': (0.0034534285239913665, 0.0004171980848481958),
 'Precision@20': (0.00319608168686701, 0.0003519296276898786),
 'Precision@5': (0.004183961480989541, 0.0007382332383653887),
 'Precision@50': (0.002596712601693508, 0.0002083550688013511),
 'R-Precision': (0.0040558836361050945, 0.0007802855920234222),
 'Recall@10': (0.008606703092025119, 0.0014433258931646248),
 'Recall@15': (0.012157606904155158, 0.001753583774714441),
 'Recall@20': (0.015070686498959892, 0.0019515171876247736),
 'Recall@5': (0.004654171867105546, 0.0010161265489507072),
 'Recall@50': (0.0314157611379

# Try different Keyphrases size

In [42]:
#Stemming and Lemmatisation
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
stem = PorterStemmer()
word = 'inversely'
print ('stemming:', stem.stem(word))
print ('lemmatization:', lem.lemmatize(word, "v"))
import re 
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

('stemming:', u'invers')
('lemmatization:', 'inversely')


In [43]:
stop_words = set(stopwords.words("english"))

In [44]:
def get_corpus(df, review_name = 'conca_review'):
    corpus = []
    for i in tqdm(range(len(df))):
        corpus.append(df[review_name][i])
    return corpus

In [46]:
# Get corpus and CountVector
from sklearn.feature_extraction.text import CountVectorizer
import re

In [47]:
corpus = get_corpus(df_train)

100%|███████████████████████████████████████████████████████████████████████| 116660/116660 [00:01<00:00, 82503.53it/s]


In [89]:
cv=CountVectorizer(max_df=0.9,stop_words=stop_words, max_features=2000, ngram_range=(1,1))
X2000=cv.fit_transform(corpus)

In [49]:
def get_I_K_fromX(df, X, row_name = 'business_num_id', binary = True, shape = (121994,6000)):
    rows = []
    cols = []
    vals = []
    for i in tqdm(range(X.shape[0])):
        arr = X[i].toarray() 
        nonzero_element = arr.nonzero()[1]  # Get nonzero element in each line
        length_of_nonzero = len(nonzero_element) 
        # df[row_name][i] is the item idex
        rows.extend(np.array([df[row_name][i]]*length_of_nonzero)) ## Item index
        cols.extend(nonzero_element) ## Keyword Index
        if binary:
            vals.extend(np.array([1]*length_of_nonzero))
        else:
            vals.extend(arr[arr.nonzero()])    
    return csr_matrix((vals, (rows, cols)), shape=shape)

In [50]:
X400

<116660x400 sparse matrix of type '<type 'numpy.int64'>'
	with 4757013 stored elements in Compressed Sparse Row format>

In [59]:
X4000

<116660x4000 sparse matrix of type '<type 'numpy.int64'>'
	with 8645321 stored elements in Compressed Sparse Row format>

In [90]:
X2000

<116660x2000 sparse matrix of type '<type 'numpy.int64'>'
	with 7659862 stored elements in Compressed Sparse Row format>

In [51]:
rtrain

<6056x4395 sparse matrix of type '<type 'numpy.float32'>'
	with 97781 stored elements in Compressed Sparse Row format>

In [91]:
I_K2000 = get_I_K_fromX(df_train,X2000, binary=False, row_name='ItemIndex',shape = (4395,4000))

100%|████████████████████████████████████████████████████████████████████████| 116660/116660 [00:18<00:00, 6272.04it/s]


In [92]:
U_K2000 = get_I_K_fromX(df_train,X2000, binary=False, row_name='UserIndex',shape = (6056,4000))

100%|████████████████████████████████████████████████████████████████████████| 116660/116660 [00:18<00:00, 6275.75it/s]


In [63]:
I_K4000

<4395x4000 sparse matrix of type '<type 'numpy.int64'>'
	with 3500695 stored elements in Compressed Sparse Row format>

In [64]:
U_K4000

<6056x4000 sparse matrix of type '<type 'numpy.int64'>'
	with 3521330 stored elements in Compressed Sparse Row format>

# Combined ?

In [95]:
## Utility functions
def add_two_matrix(ratio, U_I_matrix,I_K_matrix, shape = (3906, 3000+5791)):
    # ratio determine Keywords/User in the matrix
    rows = []
    cols = []
    datas = []
    I_U_matrix = U_I_matrix.transpose()
    
    for i in tqdm(range(I_K_matrix.shape[0])):
        nonzero1 = I_K_matrix[i].nonzero()
        nonzero2 = I_U_matrix[i].nonzero()
        row = [i]*(len(nonzero1[1])+len(nonzero2[1]))
        col = nonzero1[1].tolist()+ nonzero2[1].tolist()
        data = [ratio]*len(nonzero1[1])+[1-ratio]*len(nonzero2[1]) # Binary representation of I-K/U matrix
        
        rows.extend(row)
        cols.extend(col)
        datas.extend(data)
    return csr_matrix( (datas,(rows,cols)), shape=shape )

def transfer_to_implicit(rating_matrix, threshold = 0):
    temp_rating_matrix = sparse.csr_matrix(rating_matrix.shape)
    temp_rating_matrix[(rating_matrix > threshold).nonzero()] = 1
    rating_matrix = temp_rating_matrix
    return rating_matrix

In [97]:
rtrain

<6056x4395 sparse matrix of type '<type 'numpy.float32'>'
	with 97781 stored elements in Compressed Sparse Row format>

In [96]:
I_K_forcombine = transfer_to_implicit(I_K4000)

D:\Anaconda\envs\py27\lib\site-packages\ipykernel_launcher.py:23: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.


In [102]:
I_K_U = add_two_matrix(0.5, rtrain, I_K_forcombine, shape = (4395, 4000+6056))
similarity = train(I_K_U)
combined_0p5 = get_res(similarity,rtrain,rtest, k = 100, base = 1)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5106.24it/s]


# Different Similarity Calculation method

In [104]:
a = I_K4000

In [107]:
b = a*a.transpose()

In [108]:
b

<4395x4395 sparse matrix of type '<type 'numpy.int64'>'
	with 19272068 stored elements in Compressed Sparse Row format>

In [112]:
b[0][b[0].nonzero()]

matrix([[   29,    58,    49, ..., 98172, 50493, 54810]], dtype=int64)

In [111]:
I_K4000

<4395x4000 sparse matrix of type '<type 'numpy.int64'>'
	with 3500695 stored elements in Compressed Sparse Row format>

# Tunning

In [74]:
def get_res(similarity, rtrain, rvalid, k=100, topK = 50, I_K = None, U_K = None, base = 0):
    """
    base = 0 means regular user_item KNN
    base = 1 means keyphrases-item_based similarity KNN
    base = 2 means keyphrases-user_based similarity KNN
    """
    item_similarity_en = False
    if base == 1:
        item_similarity_en = True
    prediction_score = predict(rtrain, k, similarity, item_similarity_en= item_similarity_en)
    predict_res = prediction(prediction_score, topK, rtrain)
    res = evaluate(predict_res, rvalid)
    return res

In [93]:
def hyper_parameter_tuning(rtrain, rvalid, save_path, I_K, U_K):
    table_path = 'CDsVinyl/tables/'
    try:
        df = load_dataframe_csv(table_path, save_path)
    except:
        df = pd.DataFrame(columns=['model', 'k', 'topK'])

    num_user = rtrain.shape[0]

    for algorithm in ['item_based']:
        
        if algorithm == 'user_item':
            similarity = train(rtrain)
        elif algorithm == 'item_based':
            similarity = train(I_K)
        elif algorithm == 'user_based':
            similarity = train(U_K)
            
        for k in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000]:
            if algorithm == 'user_item':
                result =  get_res(similarity, rtrain,rvalid, k=k, base = 0)
                print "get result for k = ",k," , with algorithm = ", algorithm
            elif algorithm == 'item_based':
                result = get_res(similarity, rtrain,rvalid, k=k, I_K = I_K, base = 1)
                print "get result for k = ",k," , with algorithm = ", algorithm
            else:
                result = get_res(similarity, rtrain,rvalid, k=k, U_K = U_K, base = 2)
                print "get result for k = ",k," , with algorithm = ", algorithm

            result_dict = {'model': algorithm, 'k': k}

            for name in result.keys():
                result_dict[name] = [round(result[name][0], 4), round(result[name][1], 4)]

            df = df.append(result_dict, ignore_index=True)

            save_dataframe_csv(df, table_path, save_path)

def save_dataframe_csv(df, path, name):
    df.to_csv(path+name, index=False)

In [39]:
hyper_parameter_tuning(rtrain, rvalid, 'knn_tunning.csv', I_K, U_K)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5302.98it/s]


get result for k =  10  , with algorithm =  user_item


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5247.83it/s]


get result for k =  50  , with algorithm =  user_item


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5312.28it/s]


...

get result for k =  100  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5383.11it/s]


get result for k =  200  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5312.28it/s]


get result for k =  300  , with algorithm =  user_based


In [41]:
hyper_parameter_tuning(rtrain, rvalid, 'knn_tunning_final.csv', I_K, U_K)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4923.58it/s]


get result for k =  10  , with algorithm =  user_item


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4895.72it/s]


get result for k =  20  , with algorithm =  user_item


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4984.36it/s]


...

get result for k =  800  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5270.67it/s]


get result for k =  900  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5247.83it/s]


get result for k =  1000  , with algorithm =  user_based


In [94]:
hyper_parameter_tuning(rtrain, rvalid, 'knn_tunning_final_2000.csv', I_K2000, U_K2000)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4947.71it/s]


get result for k =  10  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4794.93it/s]


get result for k =  20  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4939.64it/s]


...

get result for k =  800  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5154.04it/s]


get result for k =  900  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4587.88it/s]


get result for k =  1000  , with algorithm =  item_based


In [76]:
hyper_parameter_tuning(rtrain, rvalid, 'knn_tunning_final_4000.csv', I_K4000, U_K4000)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4401.16it/s]


get result for k =  10  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5021.56it/s]


get result for k =  20  , with algorithm =  item_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4931.60it/s]


...

get result for k =  800  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5154.04it/s]


get result for k =  900  , with algorithm =  user_based


100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4378.89it/s]


get result for k =  1000  , with algorithm =  user_based


# Test Res

### User-item

In [79]:
similarity = train(rtrain)
user_item_test_res = get_res(similarity, rtrain,rtest, k=100 , base = 0)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4996.70it/s]


In [81]:
user_item_test_res

{'MAP@10': (0.06967576301474467, 0.003166036638077451),
 'MAP@15': (0.06299312349816566, 0.002642216310283345),
 'MAP@20': (0.05815498052932873, 0.002313510028788075),
 'MAP@5': (0.07996363035212431, 0.004219137398488782),
 'MAP@50': (0.04287378323036822, 0.0014896505793858103),
 'NDCG': (0.16056426673261637, 0.004679162839418328),
 'Precision@10': (0.05435609191601918, 0.002179272606282965),
 'Precision@15': (0.04690582465421282, 0.0017270471705763456),
 'Precision@20': (0.04169284179203174, 0.00147050452077659),
 'Precision@5': (0.06877169780128947, 0.0032115203466584427),
 'Precision@50': (0.026963134402380563, 0.0008807019950014483),
 'R-Precision': (0.06971269444272153, 0.0036516275234849916),
 'Recall@10': (0.13016474382769494, 0.005599369026932372),
 'Recall@15': (0.16674700005122858, 0.0061824760781168915),
 'Recall@20': (0.19547171410919253, 0.006590533272274745),
 'Recall@5': (0.08534234229083451, 0.004539139168287699),
 'Recall@50': (0.30009099604718237, 0.007562927041383074

### Item_based

In [80]:
similarity = train(I_K4000)
item_based_test_res = get_res(similarity, rtrain, rtest, k=10 , I_K = I_K4000 , base = 1)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 4988.47it/s]


In [86]:
item_based_test_res

{'MAP@10': (0.07178887629333984, 0.0031642553731385755),
 'MAP@15': (0.064992031749347, 0.002635286591977173),
 'MAP@20': (0.06000381513721743, 0.002304365891757811),
 'MAP@5': (0.08220697635972887, 0.0042622624420715505),
 'MAP@50': (0.044192917573934835, 0.0014837377050769267),
 'NDCG': (0.16778164007455423, 0.004908149428189497),
 'Precision@10': (0.056670524053562564, 0.002183254014644424),
 'Precision@15': (0.04838265278007384, 0.0017324994225048935),
 'Precision@20': (0.04308976690362043, 0.001466437083771068),
 'Precision@5': (0.07019342040006613, 0.003196544742899958),
 'Precision@50': (0.027554967763266658, 0.0009091317645205956),
 'R-Precision': (0.07243349225360872, 0.0037471162564621336),
 'Recall@10': (0.1419863086006407, 0.005926964704051735),
 'Recall@15': (0.17839619650584668, 0.006543351311776944),
 'Recall@20': (0.20791309653313403, 0.006932247875513412),
 'Recall@5': (0.09135255791336933, 0.004732662881220303),
 'Recall@50': (0.30698083409364585, 0.007815814255038857

### user_based

In [87]:
similarity = train(U_K4000)
user_based_test_res = get_res(similarity, rtrain, rtest, k=60 , U_K = U_K4000 , base = 2)

100%|████████████████████████████████████████████████████████████████████████████| 6056/6056 [00:01<00:00, 5055.09it/s]


In [88]:
user_based_test_res

{'MAP@10': (0.027662961475988426, 0.0020350425531479774),
 'MAP@15': (0.02533236979095799, 0.0017040315413899023),
 'MAP@20': (0.02365930147648219, 0.001495541701848737),
 'MAP@5': (0.030833746624786464, 0.0026988426807522743),
 'MAP@50': (0.018425052652704506, 0.0009706651132499718),
 'NDCG': (0.06930589317092652, 0.0031603934852285132),
 'Precision@10': (0.022648371631674658, 0.001422552135699057),
 'Precision@15': (0.01962858874745137, 0.0011206733198182316),
 'Precision@20': (0.017912051578773354, 0.0009585532836581427),
 'Precision@5': (0.028004628864275093, 0.0020950784481967824),
 'Precision@50': (0.013066622582244999, 0.0005930316778871791),
 'R-Precision': (0.026034915073542435, 0.0021865854175256334),
 'Recall@10': (0.05141660514915298, 0.0035963108493951806),
 'Recall@15': (0.06612453783626274, 0.004046048067636085),
 'Recall@20': (0.07933286856577168, 0.004426999086434319),
 'Recall@5': (0.03247410939701062, 0.002846364877967614),
 'Recall@50': (0.1397135668983857, 0.005679

### Combined

In [99]:
combined_0p3

{'MAP@10': (0.05122108599873519, 0.002653053174158122),
 'MAP@15': (0.04728615084211051, 0.002234313162919645),
 'MAP@20': (0.044302953077863805, 0.001973353451834665),
 'MAP@5': (0.05715159530500909, 0.0035125556806445434),
 'MAP@50': (0.0343378474411469, 0.0013191224541127305),
 'NDCG': (0.1291822309749353, 0.004058282726779298),
 'Precision@10': (0.04289965283517937, 0.0019288484020809802),
 'Precision@15': (0.03743869510111864, 0.0015467812090507785),
 'Precision@20': (0.0339890891056373, 0.0013391918788232815),
 'Precision@5': (0.05121507687221028, 0.002722001741904351),
 'Precision@50': (0.02368655976194413, 0.0008491465294527612),
 'R-Precision': (0.05140323281327665, 0.003060655529551258),
 'Recall@10': (0.10058189191918364, 0.004825023648988774),
 'Recall@15': (0.1302496659116421, 0.005459024789206498),
 'Recall@20': (0.15525854859961502, 0.005890230082023904),
 'Recall@5': (0.061712547308052854, 0.003772189331154954),
 'Recall@50': (0.25575726922052294, 0.007141997259463763)}

In [101]:
combined_0p7

{'MAP@10': (0.03673916979587339, 0.002286533377179637),
 'MAP@15': (0.034118915745630904, 0.0019212596961479982),
 'MAP@20': (0.03208210771095433, 0.0016927545980589594),
 'MAP@5': (0.041125805918333605, 0.003047809332245553),
 'MAP@50': (0.025521729435212723, 0.0011243306147876919),
 'NDCG': (0.09622570632715484, 0.0034663463788181885),
 'Precision@10': (0.030798479087452472, 0.0016503500760144765),
 'Precision@15': (0.0274205102771808, 0.0013242743079636638),
 'Precision@20': (0.02514465200859646, 0.0011463190745644226),
 'Precision@5': (0.03613820466192759, 0.002331153330160032),
 'Precision@50': (0.018515457100347164, 0.0007355982299518363),
 'R-Precision': (0.03640024899739321, 0.002535947262955616),
 'Recall@10': (0.06939098096256677, 0.003972644327012725),
 'Recall@15': (0.09166402392141766, 0.004539900718811375),
 'Recall@20': (0.11041799333938261, 0.004922715482932636),
 'Recall@5': (0.042286912206791864, 0.0031134462589319286),
 'Recall@50': (0.19606840268027342, 0.0063256191

In [103]:
combined_0p5

{'MAP@10': (0.039651254175555706, 0.0023419353616829044),
 'MAP@15': (0.03661492769031205, 0.0019680544909394947),
 'MAP@20': (0.03438411411806222, 0.0017331121658462432),
 'MAP@5': (0.044217225987766576, 0.0031041659709597543),
 'MAP@50': (0.027279216705910336, 0.0011602997177624112),
 'NDCG': (0.10280954981495931, 0.003574171634216811),
 'Precision@10': (0.03293106298561745, 0.0016814437954542605),
 'Precision@15': (0.02930511930346614, 0.0013568683818841788),
 'Precision@20': (0.026839146966440734, 0.0011669307215516078),
 'Precision@5': (0.03911390312448339, 0.0023881604933671404),
 'Precision@50': (0.019623078194742936, 0.0007657227552223958),
 'R-Precision': (0.03840522703948805, 0.0026043921678075422),
 'Recall@10': (0.0751278602576423, 0.004067021736928121),
 'Recall@15': (0.09940257453920837, 0.004692868904043398),
 'Recall@20': (0.11954810329703666, 0.005100662755024354),
 'Recall@5': (0.04600078165534367, 0.003243473193381281),
 'Recall@50': (0.20873636149442928, 0.006526595